# In this part i have performed Hyperparametric tuning using Optuna.

I have copied certian part of my code from other notebook and performed Tuning on it.

In [1]:
#Libraries for Numeric analysis
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts

#Libraries for Visual representation of data
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#Library to remove uncecessary waarnings messages.
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv('raw.csv', header=None)
print(df.shape)

#Convert the single row of space-separated values into multiple columns
df=pd.DataFrame(df[0].str.split(' ').tolist())

#Rename columns
column_names=['KINF', 'PPPF'] + [f'C{i}' for i in range(1, df.shape[1] - 1)]
df.columns=column_names[:df.shape[1]]


#Save the resulting DataFrame to a new CSV file named "processed.csv"
df.to_csv('processed.csv', index=False)
df_1=pd.read_csv('processed.csv')
print(df_1.shape)

df_1.drop("C40", axis=1, inplace=True)
df.to_csv('processed.csv', index=False)
print(df_1.shape)

(24000, 1)
(24000, 42)
(24000, 41)


In [4]:
#Importing necessary Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential

#These LIbraries are used for creating the CNN architecture.
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout, Input, LeakyReLU, BatchNormalization

#This Library is used to Normalize the the dataset which is part of pre-processing.
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [5]:
X=df_1.drop(columns=['PPPF'])
Z=df_1['PPPF']

#Agian This code snippet aims to select the top performing featues.
#In this case after trying many value for "num_features" 39 came out to be the best(It didnt really help).
from sklearn.feature_selection import SelectKBest, f_regression
num_features=39
selector=SelectKBest(score_func=f_regression, k=num_features)
X_selected=selector.fit_transform(X, Z)

X_train,X_test,Z_train,Z_test,=tts(X_selected,Z,test_size=0.3,random_state=64)

In [6]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# Reshape input for CNN (add a channel dimension for Conv1D).A Conv1D layer expects input data to have a 3-dimensional shape
X_train_cnn=np.expand_dims(X_train, axis=-1)
X_test_cnn=np.expand_dims(X_test, axis=-1)
print(X_train_cnn.shape)
print(X_test_cnn.shape)

(16800, 39, 1)
(7200, 39, 1)


# This was a trial run

This was the models performance Before Tuning.

In [ ]:
#L2 regularization is a technique used to prevent overfitting in machine learning models.
#Id does so by adding a penalty to the loss function based on the squared magnitude of the model's weights.(Learning from mistakes)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

#CNN model's Architecture.
model = Sequential([
        Input(shape=(X_train_cnn.shape[1], X_train_cnn.shape[2])),
        Conv1D(128, kernel_size=3,activation='relu'),
        Dropout((0.2)),
        Conv1D(64, kernel_size=3,activation='relu'),#pehale 64 tha
        Dropout((0.2)),
        Flatten(),
        Dense(64, kernel_regularizer=l2(0.01),activation='relu'),
        Dropout((0.2)),
        Dense(32,activation='relu'),
        Dense(1)
])

#Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

#Adding callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

#Training the model
history = model.fit(
    X_train_cnn, Z_train,
    validation_data=(X_test_cnn, Z_test),
    epochs=100,
    batch_size=32,
    verbose=1,
    callbacks=[early_stopping, lr_scheduler]
)

#Evaluating the model based on Mean Absolute error.
loss, mae = model.evaluate(X_test_cnn, Z_test, verbose=1)
print(f"Test Mean Absolute Error: {mae}")


Epoch 1/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - loss: 0.6097 - mae: 0.2909 - val_loss: 0.0924 - val_mae: 0.2723 - learning_rate: 0.0010
Epoch 2/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - loss: 0.0271 - mae: 0.1064 - val_loss: 0.0951 - val_mae: 0.2924 - learning_rate: 0.0010
Epoch 3/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 18s 16ms/step - loss: 0.0151 - mae: 0.0834 - val_loss: 0.0626 - val_mae: 0.2341 - learning_rate: 0.0010
Epoch 4/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - loss: 0.0109 - mae: 0.0725 - val_loss: 0.0287 - val_mae: 0.1472 - learning_rate: 0.0010
Epoch 5/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 11s 20ms/step - loss: 0.0084 - mae: 0.0652 - val_loss: 0.0163 - val_mae: 0.1047 - learning_rate: 0.0010
Epoch 6/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 18s 16ms/step - loss: 0.0080 - mae: 0.0644 - val_loss: 0.0244 - val_mae: 0.1320 - learning_rate: 0.0010
Epoch 7/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 12s 19ms/step - loss: 0.0077 - mae: 0.0632 - val_loss: 0.0142 - val_mae: 0.0952 - learnin

# Hyperparametric tuning using Optuna

In [7]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.4/383.4 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.2 MB/s eta 0:00:00


*AI use in implementing Optuna Hyperparametric Tuning*

In [ ]:
import optuna
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LeakyReLU

def objective(trial):
    # Hyperparameters to optimize
    num_filters_1 = trial.suggest_int('num_filters', 32, 200, step=32)
    num_filters_2 = trial.suggest_int('num_filters', 32, 200, step=32)
    num_filters_3 = trial.suggest_int('num_filters', 32, 200, step=32)
    kernel_size_1 = trial.suggest_int('kernel_size', 3, 7)
    kernel_size_2 = trial.suggest_int('kernel_size', 3, 7)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    dense_units = trial.suggest_int('dense_units', 64, 256, step=64)

    model = Sequential([
        Input(shape=(X_train_cnn.shape[1], X_train_cnn.shape[2])),
        Conv1D(num_filters_1, kernel_size=kernel_size_1),
        LeakyReLU(),
        Conv1D(num_filters_2, kernel_size=kernel_size_2),
        LeakyReLU(),
        Flatten(),
        Dense(dense_units, kernel_regularizer=regularizers.l2(0.01)),
        LeakyReLU(),
        Dropout(dropout_rate),
        Dense(num_filters_3),
        LeakyReLU(),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

    history = model.fit(X_train_cnn, Z_train, validation_data=(X_test_cnn, Z_test),
                        epochs=10, batch_size=64, verbose=0)

    loss, mae = model.evaluate(X_test_cnn, Z_test, verbose=0)
    return mae  # Return the evaluation metric you want to optimize

# Create study and optimize
study = optuna.create_study(direction='minimize')  # Minimize MAE
study.optimize(objective, n_trials=50)

print("Best trial:")
trial = study.best_trial
print(f"Value: {trial.value}")
for key, value in trial.params.items():
    print(f"  {key}: {value}")

[I 2025-01-26 11:48:20,388] A new study created in memory with name: no-name-d77c82f2-2e35-4271-a6c7-21c3c7cf0497
[I 2025-01-26 11:55:39,575] Trial 0 finished with value: 0.056845489889383316 and parameters: {'num_filters': 192, 'kernel_size': 6, 'dropout_rate': 0.3412511414349714, 'dense_units': 256}. Best is trial 0 with value: 0.056845489889383316.
[I 2025-01-26 11:56:52,570] Trial 1 finished with value: 0.05574087053537369 and parameters: {'num_filters': 64, 'kernel_size': 4, 'dropout_rate': 0.4707488435477307, 'dense_units': 256}. Best is trial 1 with value: 0.05574087053537369.
[I 2025-01-26 11:57:40,490] Trial 2 finished with value: 0.07907459884881973 and parameters: {'num_filters': 64, 'kernel_size': 4, 'dropout_rate': 0.2854277940777422, 'dense_units': 64}. Best is trial 1 with value: 0.05574087053537369.
[I 2025-01-26 11:58:30,074] Trial 3 finished with value: 0.057668041437864304 and parameters: {'num_filters': 64, 'kernel_size': 6, 'dropout_rate': 0.22481167554009304, 'den

I stopped further searching for better parameters as for many previous iterations the performance wasnt increasing.

# Model's performance after Tuning the model and manual tweaking.

In [ ]:
#L2 regularization is a technique used to prevent overfitting in machine learning models.
#Id does so by adding a penalty to the loss function based on the squared magnitude of the model's weights.(Learning from mistakes)
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

#CNN model's Architecture.
model = Sequential([
        Input(shape=(X_train_cnn.shape[1], X_train_cnn.shape[2])),
        Conv1D(128, kernel_size=3,activation='relu'),
        Dropout((0.2)),
        Conv1D(64, kernel_size=3,activation='relu'),#pehale 64 tha
        Dropout((0.2)),
        Flatten(),
        Dense(64, kernel_regularizer=l2(0.1),activation='relu'),
        Dropout((0.2)),
        Dense(32,activation='relu'),
        Dense(1)
])

#Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

#Adding callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

#Training the model
history = model.fit(
    X_train_cnn, Z_train,
    validation_data=(X_test_cnn, Z_test),
    epochs=100,
    batch_size=32,
    verbose=1,
    callbacks=[early_stopping, lr_scheduler]
)

#Evaluating the model based on Mean Absolute error.
loss, mae = model.evaluate(X_test_cnn, Z_test, verbose=1)
print(f"Test Mean Absolute Error: {mae}")


Epoch 1/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - loss: 2.1233 - mae: 0.2618 - val_loss: 0.0146 - val_mae: 0.0682 - learning_rate: 0.0010
Epoch 2/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - loss: 0.0281 - mae: 0.1184 - val_loss: 0.0092 - val_mae: 0.0598 - learning_rate: 0.0010
Epoch 3/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - loss: 0.0151 - mae: 0.0891 - val_loss: 0.0079 - val_mae: 0.0614 - learning_rate: 0.0010
Epoch 4/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - loss: 0.0117 - mae: 0.0781 - val_loss: 0.0102 - val_mae: 0.0698 - learning_rate: 0.0010
Epoch 5/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - loss: 0.0100 - mae: 0.0707 - val_loss: 0.0079 - val_mae: 0.0614 - learning_rate: 0.0010
Epoch 6/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 18s 16ms/step - loss: 0.0094 - mae: 0.0680 - val_loss: 0.0091 - val_mae: 0.0681 - learning_rate: 0.0010
Epoch 7/100
525/525 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - loss: 0.0092 - mae: 0.0673 - val_loss: 0.0077 - val_mae: 0.0611 - learning